In [1]:
"""
Tube location file format

Type,ID,Well
primer,P1,A1
primer,P2,A2
...
sample,S1,B1
sample,S2,B2
...

"""


'\nTube location file format\n\nType,ID,Well\nprimer,P1,A1\nprimer,P2,A2\n...\nsample,S1,B1\nsample,S2,B2\n...\n\n'

In [2]:
import pandas as pd
from opentrons import protocol_api

metadata = {
    'apiLevel': '2.13',
    'protocolName': 'Transfer Components from Separate Thermoblock Tubes to 384-Well Plate with Pandas'
}

AttributeError: module 'asyncio' has no attribute 'coroutine'

In [ ]:
def map_tube_to_well(dataframe, thermoblock, primer_tube_positions, sample_tube_positions, p20_single):
    for index, row in dataframe.iterrows():
        for col, (primer, sample) in enumerate(row):
            primer_source_well = primer_tube_positions.loc[primer, 'Well']
            sample_source_well = sample_tube_positions.loc[sample, 'Well']
            dest_well = p384.wells()[index * len(dataframe.columns) + col]

            # Transfer primer and sample to the destination well
            p20_single.transfer(2.5, primer_source_well, dest_well)
            p20_single.transfer(2.5, sample_source_well, dest_well)

In [ ]:
def run(protocol: protocol_api.ProtocolContext):
    # Labware
    primer_tubes = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap', '1')
    sample_tubes = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap', '2')
    p384 = protocol.load_labware('corning_384_wellplate_112ul_flat', '3')
    tiprack_20ul_1 = protocol.load_labware('opentrons_96_tiprack_20ul', '4')
    tiprack_20ul_2 = protocol.load_labware('opentrons_96_tiprack_20ul', '5')
    tiprack_20ul_3 = protocol.load_labware('opentrons_96_tiprack_20ul', '6')

    # Set temp to 4C
    temp_mod.set_temperature(celsius=4)
    temp_mod.status  # 'holding at target'

    temp_labware = temp_mod.load_labware(
        "opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap",
        label="Temperature-Controlled Tubes of H20",
    )

    # Pipettes
    p20_single = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tiprack_20ul_1, tiprack_20ul_2, tiprack_20ul_3])

    # Read CSV files
    transfer_data = pd.read_csv('transfer_data.csv')  # Replace with the actual file name
    tube_positions = pd.read_csv('tube_positions.csv')  # Replace with the actual file name
    primer_tube_positions = tube_positions[tube_positions['Type'] == 'primer']
    sample_tube_positions = tube_positions[tube_positions['Type'] == 'sample']

    # Set index for primer and sample tube positions
    primer_tube_positions.set_index('ID', inplace=True)
    sample_tube_positions.set_index('ID', inplace=True)

    # Transfer components
    map_tube_to_well(transfer_data, p384, primer_tube_positions, sample_tube_positions, p20_single)